In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))
source(paste0(HELP_DIR, "fisher.r"))

In [ ]:
library(cluster)
library(purrr)

# Read prepped cohorts data

- Read in prepared data and output

In [ ]:
univariate_results <- fread(paste0(SHARE_DIR, "2_run_marginal_output.csv")) 
fisher_base <- fread(paste0(SHARE_DIR, "fisher_base.csv"))
treatment_mechanism_map <- 
fread(paste0(SHARE_DIR, "treatment_mechanism_map.csv")) %>% 
 mu(derived_treatmentName = gsub( "##","/", derived_treatmentName),
    derived_treatmentMechanism = gsub( "##","/", derived_treatmentMechanism))

- Collect top independent features

In [ ]:
top <- 
univariate_results %>% 
 mu(p_fdr_fisher = p.adjust(fisher_pval, method = "fdr"), 
    p_fdr_surv = p.adjust(surv_pval, method = "fdr")) %>%
 fi(p_fdr_fisher < .05, p_fdr_surv < .05, direction == "Non-Response", !grepl("biopsy", feature), cohortGo != "Pan-Cancer")

In [ ]:
top %>% 
 gb(cohortGo) %>% 
 su(ct = n()) %>% 
 ar(desc(ct))

- What do we get

In [ ]:
library(corrplot)

In [ ]:
#top %>% fi(cohortGo == "Pan-Cancer / Immunotherapy") %>% pu(feature)

In [ ]:
corrplot(cor(
fisher_base %>% 
 #fi(cohortGo == "Pan-Cancer / Anti-PD-1") %>% 
 fi(cohortGo == "Pan-Cancer / Immunotherapy") %>%    
 #se(any_of(top %>% fi(cohortGo == "Pan-Cancer / Anti-PD-1") %>% pu(feature))), 
 se(any_of(top %>% fi(cohortGo == "Pan-Cancer / Immunotherapy") %>% pu(feature))),    
 use = "pairwise.complete.obs"
), tl.cex = .4, order = "hclust")

In [ ]:
univariate_results %>% 
 #fi(cohortGo == "Pan-Cancer / Anti-PD-1") %>%
 fi(grepl("PD", cohortGo) & ! grepl("CTLA-4", cohortGo)) %>%
 fi(feature %in% 
    c( "purity_tmbStatus_low", 
       "neo_ct_lt75",
       "rna_geneset_KEGG_RENIN_ANGIOTENSIN_SYSTEM_gt75",
       "rna_geneset_gene_set_t_cell_effector_lt50")) %>% 
 se(cohortGo, feature, or, fisher_pval, surv_est, surv_pval) %>% 
 ar(cohortGo, surv_est)

In [ ]:
table(
fisher_base %>% 
 #fi(cohortGo == "Pan-Cancer / Anti-PD-1"	) %>%
 fi( grepl( "Anti-PD-1", cohortGo )) %>%   
 se(neo_ct_lt75, 
    #purity_tmbStatus_low, 
    rna_geneset_KEGG_RENIN_ANGIOTENSIN_SYSTEM_gt50, 
    rna_geneset_gene_set_t_cell_effector_lt50,
    #rna_geneset_APM_lt50,
    non_response,
    cohortGo)
)

In [ ]:
top %>% 
 gb(cohortGo) %>% 
 su(ct = n()) %>% 
 ar(desc(ct))

# Add Clusters

- Add cluster labels for combination features

In [ ]:
auto_cluster_features <- function(data, method = "average", max_clusters = 5, corr_method = "pearson") {
  # Ensure features are columns
  if (nrow(data) < ncol(data)) {
    warning("Data has more features than samples; transposing for correlation.")
    data <- t(data)
  }
  
  # Step 1: Compute correlation matrix and convert to distance
  corr_matrix <- cor(data, method = corr_method, use = "pairwise.complete.obs")
  dist_matrix <- as.dist(1 - abs(corr_matrix))
  
  # Step 2: Hierarchical clustering
  hc <- hclust(dist_matrix, method = method)
  
  # Step 3: Evaluate silhouette scores for different cluster numbers
  best_score <- -Inf; 
  best_k <- 2; 
  best_clusters <- NULL

  for (k in 2:min(max_clusters, ncol(data) - 1)) {
    clusters <- cutree(hc, k = k)
    sil <- silhouette(clusters, dist_matrix)
    avg_sil <- mean(sil[, "sil_width"])
    
    if (avg_sil > best_score) {
      best_score <- avg_sil
      best_k <- k
      best_clusters <- clusters
    }
  }
  cat("Best number of clusters:", best_k, "\n")
  return(best_clusters)
}

In [ ]:
add_cluster_labels <- function(i = "Skin Melanoma ## Immunotherapy"){
  data <- 
  fisher_base %>% 
   fi(cohortGo == i) %>% 
   se(any_of(top %>% fi(cohortGo == i) %>% pu(feature)))

  data.frame(auto_cluster_features(data)) %>% 
   rownames_to_column("feature") %>% 
   rename(cluster = auto_cluster_features.data.) %>% 
   mu(cohortGo = i)
}

- Compute cluster labels

In [ ]:
cluster_labels <- data.frame()
for( i in unique(top$cohortGo)){
  print(i); flush.console();
  tmp <- tryCatch({add_cluster_labels(i)}, error = function(e) {return(NA)})
  if(is.data.frame(tmp)) cluster_labels <- rbind(cluster_labels, tmp)  
}

- Append cluster labels to top features

In [ ]:
top_go <- 
top %>% 
 lj(cluster_labels, by = c("cohortGo", "feature")) %>% 
 gb(cohortGo, cluster) %>% mu(rk = row_number(fisher_pval)) %>% fi(rk <= 3) %>%
 se(cohortGo, feature, cluster) %>% 
 ug() %>% 
 drop_na()

- Compute correlation for top features

In [ ]:
features <- top_go %>% fi(i == cohortGo) %>% pu(feature)
clusters <- top_go %>% fi(i == cohortGo) %>% pu(cluster)

In [ ]:
feature_pair <- list()
for( i in unique(top$cohortGo)){
    features <- top_go %>% fi(i == cohortGo) %>% pu(feature)
    clusters <- top_go %>% fi(i == cohortGo) %>% pu(cluster)
    if(length(features) > 1){
        feature_pair[[i]][["pairs"]] <- combn(features, 2, simplify = FALSE)
        feature_pair[[i]][["clusters"]] <- combn(clusters, 2, simplify = FALSE)
    } 
}

- Create index of whether features are from same cluster

In [ ]:
cluster_index <- data.frame()
for(i in names(feature_pair)){
  tmp_and <- 
  df(cohortGo = i,
     feature = unlist(lapply(feature_pair[[i]]$pairs, function(i) paste0(i[1], "_and_", i[2]))),
     clusters = unlist(lapply(feature_pair[[i]]$clusters, function(i) length(unique(i)))))

  tmp_or <- 
  df(cohortGo = i, 
     feature = unlist(lapply(feature_pair[[i]]$pairs, function(i) paste0(i[1], "_or_", i[2]))),
     clusters = unlist(lapply(feature_pair[[i]]$clusters, function(i) length(unique(i)))))
                      
  cluster_index <- rbind(cluster_index, tmp_and)
  cluster_index <- rbind(cluster_index, tmp_or)                    
}

In [ ]:
fwrite(cluster_index, paste0(SHARE_DIR, "cluster_index.csv"))

# Compute combination features

- Compute combinations for pair of features

In [ ]:
add_combination_feature <- function(i = "Skin Melanoma ## Immunotherapy", pair = c('clin_hasRadiotherapyPreTreatment','driver_B2M')){

  fisher_base %>% 
   fi( cohortGo == i) %>% 
   se( sampleId, cohortGo, non_response, pfsEvent, daysToPfsEvent, primaryTumorLocation, purity, any_of(pair)) %>% 
   mu( 
    !!paste0(pair[1], "_and_", pair[2]) := 
     case_when(
      is.na(!!sym(pair[1])) | is.na(!!sym(pair[2])) ~ NA,
      (!!sym(pair[1]) + !!sym(pair[2])) == 2 ~ 1,
      TRUE ~ 0)
  ) %>%  
  se(-any_of(pair))
}

- Add over a cohort

In [ ]:
cohort_combinations <- function(i = "Skin Melanoma ## Immunotherapy"){
  pairs <- feature_pair[[i]][['pairs']]
    
  combos <- list()
  for( j in seq(length(pairs)) ){
    pair_ready <- pairs[[j]]
    combos[[j]] <- add_combination_feature(i = i, pair = pair_ready)
  }
  ## Do separately for same or different clusters   
  clean_combos <- Filter(Negate(is.null), combos)  
  reduce(clean_combos, ~ inner_join(.x, .y, by = c("sampleId", "cohortGo", "non_response", "pfsEvent", "daysToPfsEvent", "primaryTumorLocation", "purity")))  
} 

- Collect values over cohorts

In [ ]:
combos_ready <- list()

In [ ]:
for( i in names(feature_pair)){
    print("Finding interactions: "); print(i); flush.console()
    if( !i %in% names(combos_ready)) {
      combos_ready[[i]] <- cohort_combinations(i = i)
    }
}

In [ ]:
saveRDS(combos_ready, paste0(TMP_DIR, "combo_features.Rds"))

# Compute Fisher's Exact tests

- Vamos fisher

In [ ]:
fisher_combos <- list()
for( i in names(combos_ready)){
  print(i); flush.console(); 
  fisher_combos[[i]] <- tryCatch({ra_formatter_and_test(combos_ready[[i]] %>% se(-sampleId, -pfsEvent, -daysToPfsEvent, -primaryTumorLocation, -purity))}, error = function(e){NA})
}

# Compute PFS Results

In [ ]:
scanner <- function (y = "Surv(daysToPfsEvent, pfsEvent)", features, covariates, df = "df", mod = "coxph", cohort = "Pan-Cancer") {
    out <- data.frame()
    for (f in features) {
        if (grepl("rna_", f)) {tmp <- get_stats(y = y, x = f, covariate = paste0(covariates, "+ purity"), data = df, model = mod)} 
        else {tmp <- get_stats(y = y, x = f, covariate = covariates, data = df, model = mod)}
        if (is.data.frame(tmp)) out <- rbind(out, tmp)
    }
    out
}

In [ ]:
pfs_results <- list()
for( i in names(combos_ready)){
    print(i); flush.console(); 
    tmp <- combos_ready[[i]] %>% se(-sampleId)
    categorical_features <- names(tmp %>% se(-cohortGo, -non_response, -pfsEvent, -daysToPfsEvent, -primaryTumorLocation, -purity))
    pfs_results[[i]] <- scanner(  feature = categorical_features, covariates = "", df = "tmp") %>% mu(cohortGo = i)
}

# Combine Results back together

In [ ]:
combo_results <- data.frame()
for( i in names(combos_ready)){
    print(i); flush.console();
    fisher_i <- fisher_combos[[i]]
    pfs_i <-  pfs_results[[i]]
    add <- 
    fisher_i %>% 
     lj(pfs_i %>% tm(cohortGo, feature = x, surv_est = est, surv_se = se, surv_pval = pval), 
        by = c("cohortGo", "feature")) 
    combo_results <- rbind(combo_results, add)
}

In [ ]:
combo_results_ready <- 
combo_results %>% 
 mu(type = "combination", dcb = responders, no_dcb = non_responders, ct = no_dcb + dcb) %>% 
 lj(univariate_results %>% se(cohortGo, group) %>% unique(), by = "cohortGo")

# Combine and clean results

In [ ]:
nr_threshold <- .1
fdr_threshold <- .1

In [ ]:
combiner <- function(a){
 b <- strsplit(a, "_")[[1]]
 paste0(b[-length(b)], collapse = "_")
}

In [ ]:
together <- 
rbind(
 univariate_results %>% mu(type = "univariate"), combo_results_ready) %>% 
 lj(cluster_index, by = c("cohortGo", "feature")) %>% 
 mu(clusters = ifelse(is.na(clusters), 1, clusters)) %>% 
 rw() %>% mu(short_feature = combiner(feature)) %>% ug() %>% 
 gb(cohortGo, short_feature, fisher_pval) %>% mu(tot = n()) %>% ug() %>% 
 #fi(tot == 1 | direction == "Non-Response") %>% 
 mu(or = ifelse(or == "Inf", exp(5), or), p_fdr = p.adjust(fisher_pval, method = "fdr")) %>% 
 rw() %>% mu( derived_treatmentName = str_split_fixed( cohortGo	, " / ", n = 2)[2]) %>% ug() %>% 
 lj( treatment_mechanism_map , by = "derived_treatmentName") %>% 
 mu( derived_treatmentMechanism = ifelse(is.na(derived_treatmentMechanism), derived_treatmentName, derived_treatmentMechanism),
     treatment = derived_treatmentName, mechanism = gsub(" ## ", "/", derived_treatmentMechanism),
     alpha_gp = case_when(p_fdr >= fdr_threshold ~ "none", p_fdr < nr_threshold ~ "dark"), 
     highlight = ((e_nr/events >= 1 - nr_threshold) & 
                  (p_fdr < fdr_threshold)),
     super_highlight = highlight & surv_pval < .05, 
     color_gp = case_when(highlight ~ mechanism, !highlight & (p_fdr < fdr_threshold) ~ "significant",TRUE ~ "none")) 

- Improve the Names

In [ ]:
highlights <- together %>% fi(highlight) %>% pu(feature)

In [ ]:
map1 <- 
c("_" = " " , 
  "simple" = "", 
  "clin "= "", 
  "tmbPerMb" = "TMB per Mb", #"TMB per megabase"
  "neo ct" = "RNA Predicted Neoantigens", 
  "tmlStatus" = "TML", 
  "tmbStatus" = "TMB",
  "purity " = "", 
   "hotspot KRAS position 25398284" = "KRAS G12D hotspot",     
   " gene set " = " ", 
   " geneset " = " ", 
   "gt75" = "Very High",
   "gt50" = "High",
   "gt25" = "Moderate/High",
   "lt75" = "Low/Moderate",
   "lt50" = "Low",
   "lt25" = "Very Low", 
   "cn " = "")

In [ ]:
for (i in names(map1)){
  highlights <- gsub(i, map1[[i]], highlights)
}
s2 <- str_to_title(highlights)

In [ ]:
map2 <- 
c("Kras" = "KRAS" , 
  "Rna" = "RNA", 
  "Kegg" = "",
  "Hallmark" = "",
  "And" = "and",
  "Hassystemicpretreatment" = "Systemic Pretreatment",
  "Pretreatment Contains Platinum" = "Platinum Pretreatment", 
  "T Cell" = "T-cell", 
  "Apm" = "APM", 
  "Gep" = "GEP", 
  "Cider Dna Igh" = "CIDER DNA IGH", 
  "Chrx" = "Chromosome X arm ", 
  "Tmb" = "TMB",
  "Tml" = "TML", 
  "Cd 8" = "CD8", 
  "Lt" = "Less Than ", 
  "Tgf Beta" = "TGFB",
  " Gt0" = "",
  "Antivegf" = "Anti-VEGF", 
  "  " = " ", 
  #" and " = "/", 
  "G12d" = "G12D")

In [ ]:
for (i in names(map2)) {
  s2 <- gsub(i, map2[[i]], s2 )
}

In [ ]:
share_highlights <- 
together %>% 
 fi(highlight) %>%
 #fi(super_highlight) %>%
 mu(clean_feature = s2) %>% 
 relocate(clean_feature, .after = cohortGo) %>% 
 relocate(short_feature, .after = clean_feature) %>% 
 relocate(e_r, .after = e_nr) %>%  
 relocate(p_fdr, .after = fisher_pval) %>% 
 mu(pct_response_given_event = paste0(100 * round(e_r/events,2), "%")) %>% 
 relocate(pct_response_given_event, .after = clean_feature) 

In [ ]:
together %>% fi(super_highlight) %>% gb(type) %>% su(tot = n())

In [ ]:
go <- 
together %>% 
 lj(share_highlights %>% se(cohortGo, feature, clean_feature),
    by = c("cohortGo", "feature"))

# Save results with univariate results

In [ ]:
fwrite( share_highlights, paste0(SHARE_DIR,"share_highlights_with_group.csv") )

In [ ]:
fwrite( go, paste0(SHARE_DIR, "3_run_interaction_combined_output.csv") )